In [9]:
# from nominatim_utils import get_coordinates, geocode_df
import nominatim_utils
import pandas as pd
import geopandas as gpd
import numpy as np

In [10]:
# Importar una capa con paradas de colectivos del Municipio de Tres de Febrero
url = 'https://geoportal.tresdefebrero.gob.ar/geoserver/ows?service=wfs&version=1.1.0&request=GetFeature&typename=geonode%3Aparadas_municipales'
gdf = gpd.read_file(url)
gdf = gdf.drop(columns='fin')
gdf.head()

,gml_id,fid,poligono,direc,local,intersec,estado,modelo,geometry
0,paradas_municipales.1,1,Ciudad Jardín P.M.,Aviador German Wernicke 2407,Ciudad Jardín,Alas Argentinas y Conde Zéppelin,EN EJECUCIÓN,REFUGIO PM,POINT (-58.59117 -34.59477)
1,paradas_municipales.2,2,Ciudad Jardín P.M.,Aviador German Wernicke 3099,Ciudad Jardín,Esquina Plaza del Avión,EN EJECUCIÓN,REFUGIO PM,POINT (-58.59262 -34.60243)
2,paradas_municipales.3,3,Corredor Matienzo,Aviador Teniente Benjamín Matienzo 2589,Ciudad Jardín,Agnetta y Palazzo,EN EJECUCIÓN,REFUGIO PM,POINT (-58.59454 -34.59707)
3,paradas_municipales.4,4,Ciudad Jardín P.M.,Bv. General San Martín 2943,Ciudad Jardín,De los Ceibos y De los Jacarandaes,EN EJECUCIÓN,REFUGIO PM,POINT (-58.59276 -34.60333)
4,paradas_municipales.5,5,Ciudad Jardín P.M.,Bv. General San Martín 3100,Ciudad Jardín,Rosetti y De los Aromos,EN EJECUCIÓN,BANCO,POINT (-58.59330 -34.60439)


In [11]:
# Eliminar la geometría para utilizar como ejemplo
df = pd.DataFrame(gdf.drop(columns='geometry'))
df.head()

,gml_id,fid,poligono,direc,local,intersec,estado,modelo
0,paradas_municipales.1,1,Ciudad Jardín P.M.,Aviador German Wernicke 2407,Ciudad Jardín,Alas Argentinas y Conde Zéppelin,EN EJECUCIÓN,REFUGIO PM
1,paradas_municipales.2,2,Ciudad Jardín P.M.,Aviador German Wernicke 3099,Ciudad Jardín,Esquina Plaza del Avión,EN EJECUCIÓN,REFUGIO PM
2,paradas_municipales.3,3,Corredor Matienzo,Aviador Teniente Benjamín Matienzo 2589,Ciudad Jardín,Agnetta y Palazzo,EN EJECUCIÓN,REFUGIO PM
3,paradas_municipales.4,4,Ciudad Jardín P.M.,Bv. General San Martín 2943,Ciudad Jardín,De los Ceibos y De los Jacarandaes,EN EJECUCIÓN,REFUGIO PM
4,paradas_municipales.5,5,Ciudad Jardín P.M.,Bv. General San Martín 3100,Ciudad Jardín,Rosetti y De los Aromos,EN EJECUCIÓN,BANCO


In [12]:
# Mostrar cómo se crea la consulta a la API usando la primera fila de la base
url = nominatim_utils.get_coordinates_url(country= 'Argentina', state = 'Buenos Aires', city = 'Tres de Febrero',
                        street = df.direc[1])
print(url)

http://localhost:8080/search?country=Argentina&state=Buenos Aires&city=Tres de Febrero&street=Aviador German Wernicke 3099&format=json


In [19]:
# Mostrar el primer resultado de la consulta en formato JSON
nominatim_utils.request_requests_json(url)

{'place_id': 2323179,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 28788393,
 'boundingbox': ['-34.5998449', '-34.5974207', '-58.5916724', '-58.5914696'],
 'lat': '-34.598728',
 'lon': '-58.5916056',
 'display_name': '749 - Aviador Germán Wernicke, Ciudad Jardín Lomas del Palomar, Partido de Tres de Febrero, Buenos Aires, 1685, Argentina',
 'class': 'highway',
 'type': 'tertiary',
 'importance': 0.91001}

In [20]:
# Georreferenciar la primera fila de la base. Devuelve el primer resultado de la consulta
print(nominatim_utils.get_coordinates(country= 'Argentina', state = 'Buenos Aires', city = 'Tres de Febrero',
                        street = df.direc[1]))

{'place_id': 2323179, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 28788393, 'boundingbox': ['-34.5998449', '-34.5974207', '-58.5916724', '-58.5914696'], 'lat': '-34.598728', 'lon': '-58.5916056', 'display_name': '749 - Aviador Germán Wernicke, Ciudad Jardín Lomas del Palomar, Partido de Tres de Febrero, Buenos Aires, 1685, Argentina', 'class': 'highway', 'type': 'tertiary', 'importance': 0.91001}


In [ ]:
# Georreferenciar todas las filas del data frame
df = nominatim_utils.geocode_df(df, country='Argentina', state='Buenos Aires', city='Tres de Febrero',
                                full_address_col='direc')
df.head()

In [ ]:
# Convertir a GeoDataFrame
gdf2 = gpd.GeoDataFrame(data=df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs='epsg:4326')
gdf2.head()

In [ ]:
# Graficar los puntos que logró georreferenciar
gdf2[gdf2.geometry.is_empty==False].explore()